In [ ]:
# Cell 1: Setup and Imports
import sys
import json
import pickle
from pathlib import Path

# Add scint_analysis to path if needed
pkg_root = Path("/path/to/FLITS/scintillation")  # UPDATE THIS
sys.path.insert(0, str(pkg_root))

# New modular imports
from scint_analysis import config, pipeline, widgets, plotting, analysis

print("All modules loaded successfully!")

## Step 1: Configuration

Set your burst name and paths. The rest is automatic!

In [ ]:
# Cell 2: Configuration
burst_name = "freya"  # Change to your burst name
telescope = "dsa"      # or "chime"

# Paths (update these to your system)
config_path = Path(f"configs/bursts/{burst_name}_{telescope}.yaml")
acf_cache_path = Path(f"data/cache/{burst_name}/{burst_name}_acf_results.pkl")

print(f"Burst: {burst_name}")
print(f"Config: {config_path}")
print(f"ACF cache: {acf_cache_path}")

## Step 2: Load Data and Initialize Pipeline

Load the configuration and prepare the data. This creates the `ScintillationAnalysis` pipeline object.

In [ ]:
# Cell 3: Load and Initialize
cfg = config.load_config(config_path)
print("--- Configuration Loaded ---")
print(json.dumps(cfg, indent=2))

print("\n--- Initializing Pipeline ---")
pipe = pipeline.ScintillationAnalysis(cfg)
pipe.prepare_data()

print(f"Pipeline ready! Spectrum shape: {pipe.masked_spectrum.power.shape}")

et'## Step 3: Interactive Window Selection

**NEW:** Use the interactive widget to select on/off-pulse windows!

**Before:** 110 lines of widget code in every notebook  
**After:** One function call!

**Note:** Requires `%matplotlib widget` for interactivity.

In [ ]:
# Cell 4: Window Selection (Interactive)
%matplotlib widget

# Launch the interactive window selector
widgets.interactive_window_selector(pipe, config_path)

# The widget will display automatically
# Adjust sliders and click "Save windows to YAML" when done

In [ ]:
# Cell 5: Switch back to inline plotting
%matplotlib inline

## Step 4: Run ACF Calculation

Run the pipeline to compute ACFs. Results are cached for the next step.

In [ ]:
# Cell 6: Run Pipeline
pipe.run()
print("ACF calculation complete!")

# Save ACF results for fitting
if pipe.acf_results:
    acf_cache_path.parent.mkdir(parents=True, exist_ok=True)
    with open(acf_cache_path, "wb") as f:
        pickle.dump(pipe.acf_results, f)
    print(f"ACF results saved to {acf_cache_path}")

## Step 5: Interactive ACF Fitting

**NEW:** Launch the full ACF fitting dashboard!

**Before:** 363+ lines of complex widget code  
**After:** One function call!

### Features:
- Select 1-3 model components (Lorentzian, Gaussian, Gen-Lorentz, Power-law)
- Adjust parameters with sliders
- Live plot updates (flicker-free)
- Fit with lmfit
- Save results to YAML

**Note:** Requires `%matplotlib widget`.

In [ ]:
# Cell 7: ACF Fitting Dashboard (Interactive)
%matplotlib widget

# Load cached ACF data
with open(acf_cache_path, "rb") as f:
    acf_data = pickle.load(f)

# Launch the ACF fitter dashboard
widgets.acf_fitter_dashboard(acf_data, config_path)

# The dashboard will display automatically
# Select models, adjust parameters, fit, and save!

In [ ]:
# Cell 8: Switch back to inline plotting
%matplotlib inline

## Step 6: Generate Publication Plots

**NEW:** Load saved fits and create publication-quality 3-panel plots!

**Before:** 250+ lines of plotting code  
**After:** Simple function calls!

In [ ]:
# Cell 9: Publication Plot
subband_index = 0
model_name = "Lorentzian+Gaussian"  # Change to your fitted model

# Load the saved fit from YAML
fit_data = analysis.load_saved_fit(
    config_path,
    subband_index=subband_index,
    model_name=model_name,
    lags=acf_data["subband_lags_mhz"][subband_index]
)

if fit_data:
    # Get the ACF object
    from scint_analysis.core import ACF
    acf_obj = ACF(
        lags=acf_data["subband_lags_mhz"][subband_index],
        acf=acf_data["subband_acfs"][subband_index],
        err=acf_data.get("subband_acfs_err", [None]*len(acf_data["subband_acfs"]))[subband_index]
    )
    
    # Generate the publication plot
    plotting.plot_publication_acf(
        acf_obj,
        **fit_data,
        save_path=f"plots/{burst_name}_subband{subband_index}_acf.pdf"
    )
    print(f"Publication plot generated!")
else:
    print("Could not load fit. Make sure you've run the fitting dashboard first.")

## Summary

### Code Reduction

**Old workflow (general_manual_3.ipynb):**
- **2,528 lines** of code
- 110 lines: window selector
- 363 lines: ACF fitter
- 250 lines: plotting functions
- Duplicated across 3+ notebooks

**New workflow (this notebook):**
- **~50 lines** of actual code
- Everything else is in reusable modules
- **98% reduction in notebook complexity!**

---

### Next Steps

1. **Test with your bursts**: Replace `burst_name` with your burst
2. **Customize models**: Use different component combinations
3. **Generate plots**: Create publication-ready figures
4. **Share**: This workflow works for everyone!

---

### Migration Guide

To convert your old notebooks:

1. Replace window selector code → `widgets.interactive_window_selector()`
2. Replace ACF fitter code → `widgets.acf_fitter_dashboard()`
3. Replace plotting code → `plotting.plot_publication_acf()`
4. Keep your analysis-specific code

That's it!